In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import settings
import torch
from ollama import chat, ChatResponse


c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Base de conhecimento (exemplo simples)
docs = '''Sou analista de dados, bacharel em Sistemas de Informação e atualmente curso uma
 pós-graduação em Engenharia de Machine Learning. Meu foco está na análise e
 engenharia de dados, além da automação de processos, sempre buscando
 transformar dados complexos em informações estratégicas e garantir soluções
 eficientes. Combino criatividade e pensamento analítico com senso de urgência,
 proatividade e trabalho em equipe para enfrentar desafios com foco e resiliência,
 enquanto aprimoro constantemente minhas habilidades técnicas e de liderança.
 No meu cargo atual, desenvolvo scripts em Python para criar sistemas internos, extrair
 dados via web scraping e automatizar tarefas rotineiras. Além disso, trabalho com
 SQL para manipulação eficiente de dados em um ambiente de Big Data, sempre
 buscando otimizar consultas. Com os dados refinados, crio painéis interativos e
 visualizações no Power BI e outras ferramentas, transformando conjuntos de dados
 complexos em insights acionáveis que auxiliam na tomada de decisões em diversas
 áreas do negócio.
 Python (Avançado)
 SQL (Avançado)
 Sistemas de Data Viz
 Inglês Fluente
 Metodologias Ágeis
 Docker
 Spark
 Algoritmos de
 Modelagem'''

In [ ]:
job_description = '''Job Description

Perfil de um Desenvolvedor Python OCI para migrar um Data Lake da AWS para o OCI

Desenvolvedor sênior de Python
Uso de SDKs (OCI SDK em Python)
Experiência no tratamento de dados nos formatos Parquet e JSON
Otimização de jobs Spark a nível de código (partições, joins, cache)


Serviços OCI

OCI Object Storage
OCI Data Flow
OCI SQL Endpoint
OCI SDK para Python
OCI Function
OCI ADW


Responsibilities

Descrição das atividades:

Gerenciamento de acessos a buckets/objetos.
Automação de deployments.
Desenvolvimento de scripts para copiar dados entre AWS e OCI (upload/download multipart, paralelismo).
Migração de códigos AWS para OCI.
Configuração do Data Flow.
Consultas em SQL Endpoint.
Validações pós-migração.'''

In [ ]:
def rag_query_optimized(curriculo, job_description, top_k=2):  
    # --- 2) Criar prompt otimizado ---
    prompt = f"""
    Contexto:
    Sua tarefa é ler o texto do currículo de um candidato e a descrição da vaga e gerar exatamente:

    1. Cinco pontos de destaque do candidato, em frases curtas e diretas (máximo 2 linhas cada), resumindo experiências, conquistas, habilidades e competências mais relevantes para a vaga.
    2. Cinco perguntas estratégicas para entrevista, que explorem motivação, comportamento, soft skills, competências ou requisitos da vaga que não aparecem claramente no currículo.

    Instruções obrigatórias (siga à risca):
    - Use apenas informações do currículo e da vaga. Não invente fatos.
    - Seja objetivo, claro e direto nos pontos de destaque (formato de mini resumo).
    - As perguntas devem ser diferentes entre si, relevantes para a vaga e incentivar respostas com exemplos concretos.
    - Nenhum ponto de destaque pode ser repetido.
    - Nenhuma pergunta pode ser repetida.
    - Nenhuma pergunta pode repetir informações já abordadas nos pontos de destaque.
    - Considere experiências passadas, formação acadêmica, habilidades técnicas e comportamentais.
    - Não retorne o texto de entrada, nem o contexto, nem explicações adicionais.

    - A saída deve estar **exclusivamente em formato JSON seguindo o template abaixo, sem qualquer texto fora dele:

    ```json
      "pontos_de_destaque": [
        "Resumo curto do ponto 1",
        "Resumo curto do ponto 2",
        "Resumo curto do ponto 3",
        "Resumo curto do ponto 4",
        "Resumo curto do ponto 5"
      ],
      "perguntas_entrevista": [
        "Pergunta 1",
        "Pergunta 2",
        "Pergunta 3",
        "Pergunta 4",
        "Pergunta 5"
      ]
    ```
      
    Entrada: 

    Currículo:
    {curriculo}

    Vaga:
    {job_description}

    """
    response: ChatResponse = chat(model='gemma3:4b', messages=[
        {"role": "system", "content": "Você é um assistente de recrutamento virtual."},
        {"role": "user", "content": prompt}
    ])

    return response.message.content



In [32]:
# 4) Testar
print(rag_query_optimized(job_description))

```json
{
  "pontos_de_destaque": [
    "Possui formação em Sistemas de Informação e está aprofundando seus conhecimentos em Machine Learning, demonstrando um compromisso com o aprendizado contínuo.",
    "Desenvolve scripts em Python para criação de sistemas internos e web scraping, indicando habilidades em programação e coleta de dados.",
    "Apresenta experiência em manipulação de dados em formatos Parquet e JSON, crucial para o trabalho com Data Lakes.",
    "Utiliza SQL Endpoint e Power BI para criação de painéis interativos e visualizações de dados, otimizando a análise de informações.",
    "Desenvolve e otimiza jobs Spark para a migração de dados, evidenciando conhecimento em Big Data e performance de código."
  ],
  "perguntas_entrevista": [
    "Descreva uma situação em que você precisou lidar com um problema de performance em um script ou job que estava executando. Quais foram as etapas que você tomou para identificar e resolver o problema?",
    "Como você abordaria a tare